# Fluid substitution using Gassmann
Necessary imports

In [ ]:
import matplotlib.pyplot as plt
from copy import deepcopy
import utils.io as uio
from core.well import Project, Well
from core.minerals import MineralMix
from core.fluids import FluidMix
import rp.rp_core as rp
import plotting.plot_rp as prp

## Create a project
The project remembers where important information is stored, and controls the logging. More details about how to set up a project is found in *Introduction to blixt_rp.ipnb*.

In [ ]:
wp = Project(
    name='FluidSub', 
    project_table='excels/project_table.xlsx')

Note that relative path names are allowed. The project table is a .xlsx file that contain which wells and well logs to load, where to find them, and important for fluid substitution; the fluids and minerals.

### Load wells
The *Wells table* sheet of the *project_table.xlsx* holds information about which wells and well logs to use.
<img src="images/Fluid substitution - wells.png" />
In this case, we only read in Well F, and the selected well logs that the las file *test_data/Well F.las* contains.

Wells are loaded using:

In [ ]:
wells= wp.load_all_wells(block_name='FBlock')
w = wells['WELL_F']

The *block_name* key word is used to group the logs of *Well F*, but is not necessary to specify.

### Load working intervals and templates
The working intervals are used to define where the fluid substitution will take place, and are loaded through:

In [ ]:
wis = uio.project_working_intervals(wp.project_table)
templates = uio.project_templates(wp.project_table)

### Load fluids
The *Fluids* sheet of the *project_table.xlsx* holds information about the fluids that our fluid mixtures contains.
<img src="images/Fluid substitution - fluids.png" />

Column *A* to *D* contains name, bulk and shear modolus, and density (if column *E* is set to *User specified*). 
The name of a fluid must be unique.
The option to use Batzle and Wang method to calculate the elastic fluid properties is still not fully implemented, so the *Default* fluid in this example is ignored by the system right now.

The *Fluid mixtures* sheet uses the fluids defined above to determine how they are used in fluid substitution. 
To complete a fluid substitution, it is necessary to have defined an initial and final fluid(s) for the specific well, and for the specific working interval. If you want to run fluid substitution in other wells, and/or other working intervals, in the same run, just add similar lines specifying the fluids.

<img src="images/Fluid substitution - fluidmixs.png" />

- *Column A*: **Use** 
 - The first column determines if the fluid is be included in a substitution or not
- *Column B*: **Substitution order** 
 - The second column determines if the fluid belongs to the *Initial* or *Final* fluids.
- *Column C*: **Well name**
 - Is the name of well, as defined in the *Wells table* sheet, this fluid is used in.  
- *Column D*: **Interval name**
 - name of the working interval this fluid is used
- *Column E*: **Fluid name**
 - The selected fluid, from *Fluids* sheet.
- *Column F*: **Fluid type**
 - Select if the fluid is a Brine, Gas or Oil. This will only have any meaning if the *Calculation method* of the fluid definition (*Fluids* sheet) is set to *Batzle and Wang*.
- *Column G*: **Volume fraction**
 - Column *G* determines which Volume fraction (saturation) to use for the fluid. Set it to the name of a well log (*SW* for the final Brine saturation in this example), or *complement* if the fluids takes up the remaining volume, or a number if a constant saturation is requested. If there are two or more fluids present in an intial or final fluid mix, one has to be set to *complement*.

Load the fluids through

In [ ]:
myfluids = FluidMix() 
myfluids.read_excel(wp.project_table)
print(myfluids.fluids)

### Load minerals
Similarly to the fluids, the *Minerals* sheet of the *project_table.xlsx* holds information about the minerals used in the calculation of the mineral bulk modulus
<img src="images/Fluid substitution - Minerals.png" />
Column *A* to *D* contains name, bulk and shear modolus, and density of minerals. 

The *Mineral mixtures* sheet 
<img src="images/Fluid substitution - Mineralmixs.png" />

holds information about which minerals are present in individual intervals for each well, and their volume distribution.
Similar to the fluids above. 
Column *F* determines which Volume fraction (saturation) to use for the minerals. Set it to the name of a well log (*VCL* for the Shale in this example), or *complement* if the mineral takes up the remaining volume, or a number if a constant volume fraction is requested. 

Minerals for which no *Volume fraction* is specified are ignored by the system.

**Note** that for a fluid substitution to run successfully, the fluid mixture and minral mixtures have to be properly set up in the same working intervals for the same wells in both *Fluid mixtures* and *Mineral mixtures* sheets.

Load the minerals through

In [ ]:
mymins = MineralMix() 
mymins.read_excel(wp.project_table)
print(mymins)

## Fluid substitution
Cut-offs are used to define sands, and we will restrict the fluid substitution to these

In [ ]:
cutoffs = {'Volume': ['<', 0.5], 'Porosity': ['>', 0.1]}

We also need to specify which logs we would use as inputs to the fluid substitution

In [ ]:
log_table = {'P velocity': 'vp_dry', 'S velocity': 'vs_dry', 'Density': 'rho_dry', 'Porosity': 'phie', 'Volume': 'vsh'}

The rest of the work (calculating initial and final fluid properties, mineral properties, etc.), and the gassmann fluid substitution, is handled by the following function

In [ ]:
tag = 'fs'  # tag the resulting logs after fluid substitution
rp.run_fluid_sub(wells, log_table, mymins, myfluids, cutoffs, wis, tag, block_name='FBlock')

### Plotting the results
First plot the original input logs, then replot using the newly created fluid substituted logs, and then replot again using the loaded example from an earlier fluid substitution results (from RokDoc) that was saved to the las file. 

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
log_table = {'P velocity': 'vp_dry', 'S velocity': 'vs_dry', 'Density': 'rho_dry'}
templates['WELL_F']['color'] = 'b'
prp.plot_rp(wells, log_table, wis, 'SAND E', cutoffs, templates, fig=fig, ax=ax, edge_color=False, show_masked=True, block_name='FBlock')

# specify the new logs coming from the fluid substitution
log_table = {'P velocity': 'vp_dry_fs', 'S velocity': 'vs_dry_fs', 'Density': 'rho_dry_fs'}
templates['WELL_F']['color'] = 'r'
prp.plot_rp(wells, log_table, wis, 'SAND E', cutoffs, templates, fig=fig, ax=ax, edge_color=False, block_name='FBlock')

# specify the new logs coming from the RokDoc fluid substitution stored in the las file
log_table = {'P velocity': 'vp_so08', 'S velocity': 'vs_so08', 'Density': 'rho_so08'}
templates['WELL_F']['color'] = 'g'
prp.plot_rp(wells, log_table, wis, 'SAND E', cutoffs, templates, fig=fig, ax=ax, edge_color=False, block_name='FBlock')

The grayed out points are the shales within *SAND E* working interval, and the blue markers are the initial values.
Red markers are the current fluid substitution, while the green are from an earlier fluidsubstitution in RokDoc. As can be seen does the current fluid substitution produce similar results as RocDoc.

### Saving the results
Save the well as a las file

In [ ]:
uio.write_las('results_folder/test.las', w.header, w.block['FBlock'].header, w.block['FBlock'].logs)

In [ ]:
w.block['FBlock'].logs['vp_dry_fs'].header